# Description
Dump Collection of Indonesian administrative region from BPS (Badan Pusat Statistik). There's around 82k+ rows of data

In [21]:
import pandas as pd
import requests  as re
import csv

In [22]:
def get_bps_data(level: str = 'provinsi', parent: str = '0'):
  BASE_URL = 'https://sig.bps.go.id/rest-bridging/getwilayah'
  PARAMS = { 
    'level': level,
    'parent': parent
  }
  req = re.get(url = BASE_URL, params=PARAMS)
  data = req.json()
  return data

In [23]:
OUTPATH_PROVINCE = 'output/separate/provinsi.csv'
OUTPATH_DISTRICT = 'output/separate/district.csv'
OUTPATH_REGENCY = 'output/separate/regency.csv'
OUTPATH_VILLAGE = 'output/separate/village.csv'

def get_bps_province():
  with open(OUTPATH_PROVINCE, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['kode_bps', 'nama_bps', 'kode_dagri', 'nama_dagri'])

    data_province = get_bps_data()
    for province in data_province:
      writer.writerow([province['kode_bps'],province['nama_bps'],province['kode_dagri'], province['nama_dagri']])

def get_bps_district():
  with open(OUTPATH_DISTRICT, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['parent_id', 'kode_bps', 'nama_bps', 'kode_dagri', 'nama_dagri'])
    
    data_province = pd.read_csv(OUTPATH_PROVINCE)
    for index, row in data_province.iterrows():
      province_id = row['kode_bps']
      data_district = get_bps_data(level='kabupaten', parent= int(province_id))
      for district in data_district:
        writer.writerow([province_id, district['kode_bps'], district['nama_bps'], district['kode_dagri'], district['nama_dagri']])

def get_bps_regency():
  with open(OUTPATH_REGENCY, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['parent_id', 'kode_bps', 'nama_bps', 'kode_dagri', 'nama_dagri'])
    
    data_district = pd.read_csv(OUTPATH_DISTRICT)
    for index, row in data_district.iterrows():
      district_id = row['kode_bps']
      data_regency = get_bps_data(level='kecamatan', parent= int(district_id))
      for regency in data_regency:
        writer.writerow([district_id, regency['kode_bps'], regency['nama_bps'], regency['kode_dagri'], regency['nama_dagri']])

def get_bps_villages():
  with open(OUTPATH_VILLAGE, mode='a', newline='') as file:
    writer = csv.writer(file)
    # writer.writerow(['parent_id', 'kode_bps', 'nama_bps', 'kode_dagri', 'nama_dagri'])

    data_regency = pd.read_csv(OUTPATH_REGENCY)
    for index, row in data_regency.iterrows():
      regency_id = row['kode_bps']
      if(regency_id >= 8107030):
        data_village = get_bps_data(level='desa', parent= int(regency_id))
        for village in data_village:
          writer.writerow([regency_id, village['kode_bps'],village['nama_bps'],village['kode_dagri'], village['nama_dagri']])

In [24]:
# get_bps_province()

# get_bps_district()
# 1 min 5 sec

# get_bps_regency() 
# 3 min 46 sec

get_bps_villages()
# WARNING: estimated 90 min 5sec to request data